In [20]:
import json
openapi_spec = "akoya-openapi.json"
filename = "accounts_info_get"
html_filename = f"{filename}.html"
json_filename = f"{filename}.json"
endpoint_path = "/accounts-info/{version}/{providerId}"
example_json_name = "Accounts Info Example"

def get_properties_from_ref(ref, components):
    """
    Given a $ref string, extract the actual schema it points to.
    """
    schema_name = ref.replace("#/components/schemas/", "")
    return components["schemas"][schema_name]

def merge_allOf_schemas(allOf, components):
    """
    Merge schemas under "allOf" into a single schema.
    """
    merged_schema = {"properties": {}}
    for subschema in allOf:
        if "$ref" in subschema:
            subschema = get_properties_from_ref(subschema["$ref"], components)
        if "properties" in subschema:
            merged_schema["properties"].update(subschema["properties"])
    return merged_schema

def extract_properties_from_anyOf(anyOf, components, prefix=""):
    """
    Extract properties from a given "anyOf" list.
    """
    properties = []
    for subschema in anyOf:
        properties += extract_properties(subschema, components, prefix)
    return properties

def extract_properties(schema, components, prefix=""):
    """
    Recursively extract properties from a given schema.
    """
    properties = []

    # If the schema is a dictionary of properties (common in merged schemas)
    if isinstance(schema, dict) and not any(key in schema for key in ["$ref", "allOf", "anyOf", "type"]):
        for prop_name, prop_schema in schema.items():
            full_name = f"{prefix}.{prop_name}" if prefix else prop_name
            properties += extract_properties(prop_schema, components, full_name)
        return properties

    # If the schema is a reference, dereference it
    if "$ref" in schema:
        schema = get_properties_from_ref(schema["$ref"], components)

    # If the schema has "allOf", extract properties from each of the schemas within
    if "allOf" in schema:
        merged_schema = merge_allOf_schemas(schema["allOf"], components)
        properties += extract_properties(merged_schema, components, prefix)

    # If the schema has "anyOf", extract properties from each of the schemas within
    elif "anyOf" in schema:
        properties += extract_properties_from_anyOf(schema["anyOf"], components, prefix)

    else:
        schema_type = schema.get("type")

        # If the schema is of type object and has properties, iterate over them
        if schema_type == "object" and "properties" in schema:
            for prop_name, prop_schema in schema["properties"].items():
                full_name = f"{prefix}.{prop_name}" if prefix else prop_name
                properties += extract_properties(prop_schema, components, full_name)
        elif schema_type == "array" and "items" in schema:
            properties += extract_properties(schema["items"], components, prefix)
        else:
            prop_details = {
                "property_name": prefix,
                "property_type": schema_type,
                "property_description": schema.get("description", "N/A"),
                "enum": schema.get("enum", []),
                "nullable": schema.get("nullable", False)
            }
            properties.append(prop_details)

    return properties


# Load the OpenAPI JSON file
with open(openapi_spec, "r") as file:
    openapi_data = json.load(file)


response_schema = openapi_data["paths"][endpoint_path]["get"]["responses"]["200"]["content"]["application/json"]["schema"]
properties_list = extract_properties(response_schema, openapi_data["components"])
for item in properties_list:
    if('property_name' in item.keys()):
        item["property_name"] = item['property_name'].replace("properties.", "")

# properties_list

def get_example_response_for_endpoint(openapi_data, endpoint):
    """
    Retrieves an example JSON response for a given endpoint from the OpenAPI data.
    """
    try:
        # Iterate over HTTP verbs to find the correct endpoint data
        for verb, endpoint_data in openapi_data["paths"][endpoint].items():
            if "responses" in endpoint_data and "200" in endpoint_data["responses"]:
                example_response = endpoint_data["responses"]["200"]["content"]["application/json"]["examples"][example_json_name]["value"]
                return example_response
    except KeyError:
        return None

example_json = get_example_response_for_endpoint(openapi_data, endpoint_path)

# def get_example_for_property(example_json, property_path):
#     """
#     Extracts an example value for a given property path from the example json.
#     """
#     parts = property_path.split('.')
#
#     def recursive_search(data, remaining_parts):
#         if not remaining_parts:
#             return data
#         key = remaining_parts[0]
#         if isinstance(data, list):
#             for item in data:
#                 try:
#                     result = recursive_search(item[key], remaining_parts[1:])
#                     if result is not None:
#                         return result
#                 except (KeyError, TypeError):
#                     continue
#         else:
#             try:
#                 return recursive_search(data[key], remaining_parts[1:])
#             except (KeyError, TypeError):
#                 return None
#         return None
#
#     return recursive_search(example_json, parts)

def get_example_for_property(example_json, property_path):
    """
    Extracts an example value for a given property path from the example json.
    """
    parts = property_path.split('.')

    def recursive_search(data, remaining_parts):
        if not remaining_parts:
            return data
        key = remaining_parts[0]
        if isinstance(data, list):
            for item in data:
                try:
                    result = recursive_search(item[key], remaining_parts[1:])
                    if result is not None:
                        return result
                except (KeyError, TypeError):
                    continue
        else:
            try:
                return recursive_search(data[key], remaining_parts[1:])
            except (KeyError, TypeError):
                return None
        return None

    return recursive_search(example_json, parts)

def append_examples_to_properties(properties, example_json):
    """
    Appends an example to each property in the list of properties based on the example json.
    """
    for prop in properties:
        prop["example"] = get_example_for_property(example_json, prop["property_name"])
    return properties

# Add example data to properties
properties_with_examples = append_examples_to_properties(properties_list, example_json)

from IPython.display import display
import pandas as pd
def display_as_table(properties_list):
    """
    Display a list of properties as a table in a Jupyter notebook cell.
    """
    # Convert the properties list to a DataFrame
    df = pd.DataFrame(properties_list)

    # Display the DataFrame as a table in the Jupyter notebook cell
    display(df)


def save_to_html(properties_list, filename):
    """
    Render a list of properties as an HTML table and save to a file.
    """
    # Convert the properties list to a DataFrame
    df = pd.DataFrame(properties_list)

    # Save the DataFrame as an HTML file
    df.to_html(filename, index=False, escape=False)

# Save the properties list as an HTML file
save_to_html(properties_with_examples, html_filename)

def write_properties_to_file(properties, filename):
    """
    Writes the properties list to a file.
    """
    with open(filename, 'w') as f:
        json.dump(properties, f, indent=4)

def read_properties_from_file(filename):
    """
    Reads the properties list from a file.
    """
    with open(filename, 'r') as f:
        properties = json.load(f)
    return properties

# Testing the functions
write_properties_to_file(properties_with_examples, json_filename)
loaded_properties = read_properties_from_file(json_filename)

display_as_table(properties_with_examples)

accounts.depositAccount.accountId
accounts.depositAccount.accountType
accounts.depositAccount.accountNumberDisplay
accounts.depositAccount.currency.currencyCode
accounts.depositAccount.currency.currencyRate
accounts.depositAccount.currency.originalCurrencyCode
accounts.depositAccount.description
accounts.depositAccount.fiAttributes.name
accounts.depositAccount.fiAttributes.value
accounts.depositAccount.nickname
accounts.depositAccount.productName
accounts.depositAccount.status
accounts.depositAccount.lineOfBusiness
accounts.depositAccount.balanceType
accounts.depositAccount.interestRate
accounts.depositAccount.interestRateType
accounts.depositAccount.interestRateAsOf
accounts.depositAccount.lastActivityDate
accounts.depositAccount.micrNumber
accounts.depositAccount.parentAccountId
accounts.depositAccount.priorInterestRate
accounts.depositAccount.transferIn
accounts.depositAccount.transferOut
accounts.depositAccount.annualPercentageYield
accounts.depositAccount.term
accounts.depositAcco

,property_name,property_type,property_description,enum,nullable,example
0,accounts.depositAccount.accountId,string,Long-term persistent identity of the account. ...,[],False,g833202fb0866d0ad83472c429
1,accounts.depositAccount.accountType,string,"The type of an account. For instance, CHECKING...",[],False,CHECKING
2,accounts.depositAccount.accountNumberDisplay,string,Account display number for the end user’s hand...,[],False,xxxxxxxx0071
3,accounts.depositAccount.currency.currencyCode,string,Iso 4217 currency code.,[],False,USD
4,accounts.depositAccount.currency.currencyRate,number,Currency rate between original and converted c...,[],False,None
...,...,...,...,...,...,...
153,accounts.annuityAccount.transferIn,boolean,Account is eligible for incoming transfers,[],False,None
154,accounts.annuityAccount.transferOut,boolean,Account is eligible for outgoing transfers,[],False,None
155,accounts.annuityAccount.annuityProductType,string,N/A,"[CURRENCY, SHARES]",False,None
156,accounts.annuityAccount.annuityValueBasis,string,N/A,"[FIXED, VARIABLE]",False,None
